In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/non-fulfillment/sample_submission.csv
/kaggle/input/non-fulfillment/train.csv
/kaggle/input/non-fulfillment/test.csv


In [2]:
!pip install -q pytorch-tabnet tab-transformer-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

# PyTorch 관련 라이브러리
import torch
import torch.nn as nn
import torch.optim as optim

# TabNet (pytorch-tabnet)
from pytorch_tabnet.tab_model import TabNetClassifier

# TabTransformer (tab-transformer-pytorch)
from tab_transformer_pytorch import TabTransformer

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
    

# 딥러닝 모델 (TensorFlow)
import tensorflow as tf
from tensorflow import keras


In [5]:
# # 데이터 로드 및 전처리
# train_data = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
# test_data = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')

# # UID 컬럼 제거
# train_data.drop(columns=['UID'], inplace=True)
# test_data.drop(columns=['UID'], inplace=True)

In [6]:
# 1. 데이터 로드 및 UID 제거
train_data = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
test_data = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')
train_df = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
test_df = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')
train_data.drop(columns=['UID'], inplace=True)
test_data.drop(columns=['UID'], inplace=True)


In [7]:
train_data.head()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767,0
1,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767,0
2,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800,1
3,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796,1
4,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751,0


In [8]:
test_data.head()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
0,월세,1560090.0,10년 이상,0.0,13,12.0,495561.0,0,18,0,기타,단기 상환,376332.0,133522.5,29641.5,736
1,주택 담보 대출 (거주 중),2102616.0,2년,0.0,9,29.0,580833.0,0,40,0,부채 통합,장기 상환,830379.0,302983.5,20151.0,718
2,주택 담보 대출 (거주 중),2477989.5,10년 이상,0.0,11,26.5,995841.0,0,44,0,부채 통합,장기 상환,877635.0,379278.0,13113.0,722
3,주택 담보 대출 (거주 중),1571091.0,6년,0.0,7,34.4,601656.0,0,45,0,부채 통합,단기 상환,487278.0,275395.5,11679.0,762
4,주택 담보 대출 (거주 중),2290260.0,10년 이상,0.0,19,25.0,1954623.0,0,14,0,부채 통합,단기 상환,397782.0,742767.0,42370.5,775


In [9]:
print(train_data.shape[0])
print(test_data.shape[0])

10000
2062


In [10]:
# # train_data가 이미 DataFrame 형태로 있다고 가정합니다.
# # 수치형 변수들의 상관관계 계산 (문자형 변수는 자동으로 제외됨)
# corr_matrix = train_data.corr()
# print(corr_matrix)

# # 히트맵 시각화
# plt.figure(figsize=(12, 10))
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title("Train Data - Correlation Matrix")
# plt.show()

In [11]:
print(train_data.columns)

Index(['주거 형태', '연간 소득', '현재 직장 근속 연수', '체납 세금 압류 횟수', '개설된 신용계좌 수',
       '신용 거래 연수', '최대 신용한도', '신용 문제 발생 횟수', '마지막 연체 이후 경과 개월 수', '개인 파산 횟수',
       '대출 목적', '대출 상환 기간', '현재 대출 잔액', '현재 미상환 신용액', '월 상환 부채액', '신용 점수',
       '채무 불이행 여부'],
      dtype='object')


In [12]:
print(test_df.columns)

Index(['UID', '주거 형태', '연간 소득', '현재 직장 근속 연수', '체납 세금 압류 횟수', '개설된 신용계좌 수',
       '신용 거래 연수', '최대 신용한도', '신용 문제 발생 횟수', '마지막 연체 이후 경과 개월 수', '개인 파산 횟수',
       '대출 목적', '대출 상환 기간', '현재 대출 잔액', '현재 미상환 신용액', '월 상환 부채액', '신용 점수'],
      dtype='object')


In [13]:
# 2. 범주형 변수 처리 (라벨 인코딩 & 원-핫 인코딩)
label_enc = LabelEncoder()
train_data["현재 직장 근속 연수"] = label_enc.fit_transform(train_data["현재 직장 근속 연수"])
test_data["현재 직장 근속 연수"] = label_enc.transform(test_data["현재 직장 근속 연수"])

train_data = pd.get_dummies(train_data, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)
test_data = pd.get_dummies(test_data, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)

# 3. 수치형 변수 변환 (로그 변환 및 "연체 없음" 컬럼 추가)
log_columns = ["현재 미상환 신용액", "월 상환 부채액", "현재 대출 잔액"]
for col in log_columns:
    train_data[col] = np.log1p(train_data[col])
    test_data[col] = np.log1p(test_data[col])

train_data["연체 없음"] = (train_data["마지막 연체 이후 경과 개월 수"] == 0).astype(int)
test_data["연체 없음"] = (test_data["마지막 연체 이후 경과 개월 수"] == 0).astype(int)

# 4. 파생 변수 생성
# 4. 파생 변수 생성 (분모에 epsilon 추가)
epsilon = 1e-8  # 매우 작은 상수

# DTI = (월 상환 부채액 * 12) / 연간 소득
train_data['DTI'] = (train_data['월 상환 부채액'] * 12) / (train_data['연간 소득'] + epsilon)
test_data['DTI'] = (test_data['월 상환 부채액'] * 12) / (test_data['연간 소득'] + epsilon)

# DSR = (현재 대출 잔액 + 현재 미상환 신용액) / 연간 소득
train_data['DSR'] = (train_data['현재 대출 잔액'] + train_data['현재 미상환 신용액']) / (train_data['연간 소득'] + epsilon)
test_data['DSR'] = (test_data['현재 대출 잔액'] + test_data['현재 미상환 신용액']) / (test_data['연간 소득'] + epsilon)

# LTV = (현재 대출 잔액) / 최대 신용한도
train_data['LTV'] = train_data['현재 대출 잔액'] / (train_data['최대 신용한도'] + epsilon)
test_data['LTV'] = test_data['현재 대출 잔액'] / (test_data['최대 신용한도'] + epsilon)

# 5. 무한대 값을 np.nan으로 치환
train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
test_data.replace([np.inf, -np.inf], np.nan, inplace=True)


# 5. 결측값 처리 (SimpleImputer: 중간값으로 대체)
target = "채무 불이행 여부"
train_features = train_data.drop(columns=[target])
train_target = train_data[target]

imputer = SimpleImputer(strategy="median")
train_features_imputed = pd.DataFrame(imputer.fit_transform(train_features), 
                                        columns=train_features.columns)
test_data_imputed = pd.DataFrame(imputer.transform(test_data), 
                                 columns=test_data.columns)

# 6. 데이터 스케일링 (StandardScaler 적용)
scaler = StandardScaler()
train_features_scaled = pd.DataFrame(scaler.fit_transform(train_features_imputed), 
                                     columns=train_features_imputed.columns)
test_data_scaled = pd.DataFrame(scaler.transform(test_data_imputed), 
                                columns=test_data_imputed.columns)

# 7. 불균형 데이터 처리 (SMOTE 적용)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(train_features_scaled, train_target)
train_data_final = pd.concat([pd.DataFrame(X_resampled, columns=train_features_scaled.columns), 
                              pd.DataFrame(y_resampled, columns=[target])], axis=1)

# 8. 타깃 및 피처 분리 및 train/validation 분할
label = target
X = train_data_final.drop(columns=[label]).values
y = train_data_final[label].values.astype(np.float32)
X_test = test_data_scaled.values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# # Fold별 Accuracy와 Loss 저장용 리스트 초기화
# all_history = []  # 이 줄을 추가하여 all_history를 명확히 초기화합니다.

# kf = KFold(n_splits=5, shuffle=True, random_state=42)
# fold = 1
# roc_auc_scores = []

# for train_index, valid_index in kf.split(X_resampled):
#     print(f"🔄 Fold {fold} 시작")

#     # 학습/검증 데이터 분리
#     X_train_fold, X_valid_fold = X_resampled[train_index], X_resampled[valid_index]
#     y_train_fold, y_valid_fold = y_resampled[train_index], y_resampled[valid_index]

#     # 모델 정의
#     model = keras.Sequential([
#         keras.layers.Dense(128, activation="relu", input_shape=(X_train_fold.shape[1],)),
#         keras.layers.Dropout(0.3),
#         keras.layers.Dense(64, activation="relu"),
#         keras.layers.Dropout(0.3),
#         keras.layers.Dense(32, activation="relu"),
#         keras.layers.Dense(1, activation="sigmoid")
#     ])

#     # 모델 컴파일
#     model.compile(
#         optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
#         loss="binary_crossentropy",
#         metrics=["accuracy"]
#     )

#     # 조기 종료 콜백 설정
#     early_stopping = keras.callbacks.EarlyStopping(
#         monitor="val_loss",
#         patience=5,
#         min_delta=0.001,
#         restore_best_weights=True
#     )

#     # 모델 학습
#     history = model.fit(
#         X_train_fold, y_train_fold,
#         validation_data=(X_valid_fold, y_valid_fold),
#         epochs=100,
#         batch_size=32,
#         callbacks=[early_stopping],
#         verbose=1
#     )

#     # Fold의 학습 결과 저장
#     all_history.append(history.history)  # Fold별 학습 결과 저장

#     # 검증 데이터 예측 및 평가
#     y_valid_pred = model.predict(X_valid_fold).flatten()
#     roc_auc = roc_auc_score(y_valid_fold, y_valid_pred)
#     print(f"Fold {fold} Validation ROC-AUC: {roc_auc:.4f}")
#     roc_auc_scores.append(roc_auc)
#     fold += 1

# # 교차검증 평균 결과 출력
# print(f"✅ 교차검증 평균 ROC-AUC: {np.mean(roc_auc_scores):.4f}")

# # 시각화: Fold별 Accuracy와 Loss
# for i, history in enumerate(all_history):
#     epochs = range(1, len(history['loss']) + 1)

#     plt.figure(figsize=(12, 4))

#     # Accuracy 시각화
#     plt.subplot(1, 2, 1)
#     plt.plot(epochs, history['accuracy'], label='Train Accuracy')
#     plt.plot(epochs, history['val_accuracy'], label='Validation Accuracy')
#     plt.title(f'Fold {i + 1} Accuracy')
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy')
#     plt.legend()

#     # Loss 시각화
#     plt.subplot(1, 2, 2)
#     plt.plot(epochs, history['loss'], label='Train Loss')
#     plt.plot(epochs, history['val_loss'], label='Validation Loss')
#     plt.title(f'Fold {i + 1} Loss')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()

#     plt.tight_layout()
#     plt.show()

In [15]:
# y_test_pred = model.predict(test_scaled).flatten()  # 확률 값 그대로 사용

# # 제출 파일 생성
# submission = pd.DataFrame({
#     "UID": test_uid["UID"],
#     "채무 불이행 확률": y_test_pred  # 확률값을 그대로 저장
# })

# # CSV 저장
# submission.to_csv("submission_17th.csv", index=False)

# print("✅ 제출 파일 생성 완료! 'submission_17th.csv' 저장되었습니다.")

In [16]:
train_data.head()

,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,...,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_장기 상환,연체 없음,DTI,DSR,LTV
0,1941337.5,0,0.0,9,13.4,400597.5,0,24,1,12.876217,...,False,False,False,False,False,False,0,0.000056,0.000013,0.000032
1,1979505.0,0,0.0,5,15.1,360679.5,0,11,0,13.817694,...,False,False,False,False,False,False,0,0.000061,0.000013,0.000038
2,1356381.0,5,0.0,12,18.8,491770.5,1,74,3,12.336118,...,False,False,False,False,False,False,0,0.000083,0.000019,0.000025
3,1049017.5,7,0.0,15,14.8,411546.0,1,22,1,12.434739,...,False,False,False,False,False,False,0,0.000119,0.000024,0.000030
4,4320217.5,3,0.0,11,26.1,895288.5,0,32,0,13.966666,...,False,False,False,False,False,True,0,0.000024,0.000006,0.000016


In [17]:
test_data.head()

,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,...,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_장기 상환,연체 없음,DTI,DSR,LTV
0,1560090.0,0,0.0,13,12.0,495561.0,0,18,0,12.838230,...,False,False,False,False,False,False,0,0.000079,0.000016,0.000026
1,2102616.0,3,0.0,9,29.0,580833.0,0,40,0,13.629639,...,False,False,False,False,False,True,0,0.000057,0.000012,0.000023
2,2477989.5,0,0.0,11,26.5,995841.0,0,44,0,13.684987,...,False,False,False,False,False,True,0,0.000046,0.000011,0.000014
3,1571091.0,7,0.0,7,34.4,601656.0,0,45,0,13.096592,...,False,False,False,False,False,False,0,0.000072,0.000016,0.000022
4,2290260.0,0,0.0,19,25.0,1954623.0,0,14,0,12.893662,...,False,False,False,False,False,False,0,0.000056,0.000012,0.000007


In [18]:
# 9. TabNet 모델 학습
tabnet_clf = TabNetClassifier(seed=42, verbose=1, optimizer_params=dict(lr=2e-2))
tabnet_clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['auc'],
    max_epochs=100,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128
)
tabnet_val_preds = tabnet_clf.predict_proba(X_val)[:, 1]
tabnet_auc = roc_auc_score(y_val, tabnet_val_preds)
print("TabNet Validation AUC:", tabnet_auc)

epoch 0  | loss: 0.75861 | val_0_auc: 0.60252 |  0:00:01s
epoch 1  | loss: 0.69251 | val_0_auc: 0.64746 |  0:00:01s
epoch 2  | loss: 0.66876 | val_0_auc: 0.67026 |  0:00:02s
epoch 3  | loss: 0.65539 | val_0_auc: 0.67906 |  0:00:02s
epoch 4  | loss: 0.64737 | val_0_auc: 0.69239 |  0:00:03s
epoch 5  | loss: 0.641   | val_0_auc: 0.70078 |  0:00:03s
epoch 6  | loss: 0.63815 | val_0_auc: 0.70915 |  0:00:04s
epoch 7  | loss: 0.63262 | val_0_auc: 0.71248 |  0:00:04s
epoch 8  | loss: 0.63058 | val_0_auc: 0.70929 |  0:00:05s
epoch 9  | loss: 0.63164 | val_0_auc: 0.70937 |  0:00:05s
epoch 10 | loss: 0.63033 | val_0_auc: 0.71198 |  0:00:05s
epoch 11 | loss: 0.622   | val_0_auc: 0.72133 |  0:00:06s
epoch 12 | loss: 0.6211  | val_0_auc: 0.72016 |  0:00:06s
epoch 13 | loss: 0.61927 | val_0_auc: 0.72284 |  0:00:07s
epoch 14 | loss: 0.6187  | val_0_auc: 0.72421 |  0:00:07s
epoch 15 | loss: 0.61627 | val_0_auc: 0.72932 |  0:00:08s
epoch 16 | loss: 0.61313 | val_0_auc: 0.72641 |  0:00:08s
epoch 17 | los

In [19]:
# 10. TabTransformer 모델 구현 및 학습
class TabTransformerClassifier(nn.Module):
    def __init__(self, num_continuous, num_classes=1, dim=32, depth=6, heads=8, mlp_hidden_mults=(4,), dropout=0.1):
        super().__init__()
        # 범주형 피처가 없으므로 빈 텐서를 전달합니다.
        self.tabtransformer = TabTransformer(
            categories=(), 
            num_continuous=num_continuous, 
            dim=dim,
            depth=depth,
            heads=heads,
            attn_dropout=dropout,
            ff_dropout=dropout,
            mlp_hidden_mults=mlp_hidden_mults
        )
        # 출력 차원 확장을 위한 선형 변환 레이어
        self.proj = nn.Linear(1, dim)
        self.fc = nn.Linear(dim, num_classes)
        
    def forward(self, x):
        x_cat = torch.empty(x.size(0), 0, dtype=torch.long, device=x.device)
        x = self.tabtransformer(x_cat, x)
        x = self.proj(x)
        return self.fc(x)

def train_tabtransformer(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=256, lr=1e-3, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss()  # 로짓 출력에 적합한 손실 함수
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                                                     torch.tensor(y_train, dtype=torch.float32))
    val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val, dtype=torch.float32),
                                                   torch.tensor(y_val, dtype=torch.float32))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    best_val_auc = 0
    best_state = None
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            outputs = model(xb).squeeze(1)
            loss = criterion(outputs, yb)
            loss.backward()
            optimizer.step()
        
        # 검증 단계
        model.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                outputs = model(xb).squeeze(1)
                preds = torch.sigmoid(outputs)
                all_preds.append(preds.cpu().numpy())
                all_targets.append(yb.cpu().numpy())
        all_preds = np.concatenate(all_preds)
        all_targets = np.concatenate(all_targets)
        curr_auc = roc_auc_score(all_targets, all_preds)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Val AUC: {curr_auc:.4f}")
        if curr_auc > best_val_auc:
            best_val_auc = curr_auc
            best_state = model.state_dict()
    model.load_state_dict(best_state)
    return model

num_features = X_train.shape[1]
tabtrans_model = TabTransformerClassifier(num_continuous=num_features, dim=32, depth=6, heads=8, mlp_hidden_mults=(4,), dropout=0.1)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tabtrans_model = train_tabtransformer(tabtrans_model, X_train, y_train, X_val, y_val, epochs=50, batch_size=256, lr=1e-3, device=device)

tabtrans_model.eval()
with torch.no_grad():
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    outputs = tabtrans_model(X_val_tensor).squeeze(1)
    tabtrans_val_preds = torch.sigmoid(outputs).cpu().numpy()
tabtrans_auc = roc_auc_score(y_val, tabtrans_val_preds)
print("TabTransformer Validation AUC:", tabtrans_auc)

Epoch 1/50, Loss: 0.6170, Val AUC: 0.7325
Epoch 2/50, Loss: 0.5899, Val AUC: 0.7405
Epoch 3/50, Loss: 0.6172, Val AUC: 0.7464
Epoch 4/50, Loss: 0.5851, Val AUC: 0.7509
Epoch 5/50, Loss: 0.5529, Val AUC: 0.7537
Epoch 6/50, Loss: 0.6436, Val AUC: 0.7573
Epoch 7/50, Loss: 0.6888, Val AUC: 0.7592
Epoch 8/50, Loss: 0.5292, Val AUC: 0.7621
Epoch 9/50, Loss: 0.6575, Val AUC: 0.7648
Epoch 10/50, Loss: 0.5572, Val AUC: 0.7641
Epoch 11/50, Loss: 0.5862, Val AUC: 0.7674
Epoch 12/50, Loss: 0.6351, Val AUC: 0.7683
Epoch 13/50, Loss: 0.5293, Val AUC: 0.7637
Epoch 14/50, Loss: 0.4603, Val AUC: 0.7696
Epoch 15/50, Loss: 0.5715, Val AUC: 0.7686
Epoch 16/50, Loss: 0.6182, Val AUC: 0.7722
Epoch 17/50, Loss: 0.5174, Val AUC: 0.7721
Epoch 18/50, Loss: 0.5019, Val AUC: 0.7712
Epoch 19/50, Loss: 0.5418, Val AUC: 0.7703
Epoch 20/50, Loss: 0.5158, Val AUC: 0.7720
Epoch 21/50, Loss: 0.6209, Val AUC: 0.7716
Epoch 22/50, Loss: 0.5648, Val AUC: 0.7694
Epoch 23/50, Loss: 0.6382, Val AUC: 0.7708
Epoch 24/50, Loss: 0

In [20]:
# 11. 단순 평균 앙상블
# TabNet과 TabTransformer의 예측을 단순 평균으로 결합
tabnet_test_preds = tabnet_clf.predict_proba(X_test)[:, 1]
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    outputs = tabtrans_model(X_test_tensor).squeeze(1)
    tabtrans_test_preds = torch.sigmoid(outputs).cpu().numpy()

ensemble_preds = (tabnet_test_preds + tabtrans_test_preds) / 2.0


In [21]:
# 12. 제출 파일 생성
submission = pd.read_csv('/kaggle/input/non-fulfillment/sample_submission.csv')
submission['채무 불이행 확률'] = ensemble_preds
submission.to_csv('submission_stacking.csv', index=False)
print("스태킹 앙상블을 이용한 예측 및 제출 파일 생성 완료!")

스태킹 앙상블을 이용한 예측 및 제출 파일 생성 완료!
